## Step 1 Co-registration

(1) We processed DEMs co-registration tile by tile, and get all results in .csv, which you can access from [Zenodo](https://zenodo.org/records/10048875).

(2) The alignment with ERA5 Land is done in separated notebook.

Here we start from Step 2 bias correction.





## Step 2 Bias correction

This step includes the regression and the 

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

# all function has been moduled in to xsnow
from xsnow.goregression import Snow_Regressor

# load the dataset. You might need to change the path/file name
sf = r'\\hypatia.uio.no\lh-mn-geofag-felles\projects\snowdepth\zhihaol\snow_free_.csv'
sc = r'\\hypatia.uio.no\lh-mn-geofag-felles\projects\snowdepth\zhihaol\snow_cover_.csv'
dems = Snow_Regressor(sf,sc)

# basic configuration
dems.sc['flag'] = 'sc'
dems.sf['flag'] = 'sf'
dems.era = 'sde_era'

# Using the geosegment
dems.use_geosegment()

In [ ]:
# Start regression

# setting the features for regression
v_x_dtm1=['cloud_flag_atm','subset_te_flag', 'N', 'E','h_te_std','h_te_skew','terrain_slope',
          'coreg_bias_dtm1',
          'h_te_best_fit','pair','beam','slope', 'aspect', 'planc','profc','tpi','tpi_9','tpi_27','curvature',
          'urban_flag', 'night_flag','region','difference']

v_x_dtm10=['cloud_flag_atm','subset_te_flag', 'N', 'E','h_te_std','h_te_skew','terrain_slope',
          'coreg_bias_dtm10',
          'h_te_best_fit','pair','beam','slope', 'aspect', 'planc','profc','tpi','tpi_9','tpi_27','curvature',
          'urban_flag', 'night_flag','region','difference']
                            
v_x_cop30=['cloud_flag_atm','subset_te_flag', 'N', 'E', 'h_te_skew','h_te_std','terrain_slope',
          'coreg_bias_cop30',
          'h_te_best_fit','pair','beam','slope', 'aspect', 'planc','profc','tpi','tpi_9','tpi_27','curvature',
          'segment_cover', 'urban_flag', 'night_flag','region','difference']

v_x_fab=['cloud_flag_atm','subset_te_flag', 'N', 'E', 'h_te_skew','h_te_std','terrain_slope',
          'coreg_bias_fab',
          'h_te_best_fit','pair','beam','slope', 'aspect', 'planc','profc','tpi','tpi_9','tpi_27','curvature',
          'segment_cover', 'urban_flag', 'night_flag','region','difference']
      
# We do only send the elevation difference less than +-10 m and snow free condition for processing.

dems.sf_qc = {'dtm1':' (-10<dh_after_dtm1 < 10) & brightness_flag == 0',
              'dtm10':'(-10<dh_after_dtm10 < 10) & brightness_flag == 0',
              'cop30':'(-10<dh_after_cop30 < 10) & brightness_flag == 0',
              'fab':'(-10<dh_after_fab < 10) & brightness_flag == 0'}

# settubf paraneters for processing
dems.params = {
                'objective': 'reg:squarederror',
                'n_estimators': 350,
                'max_depth': 10,
                'learning_rate': 0.1, # the lower, the slower, the roboster
                'min_child_weight': 1,
                'subsample': 0.7,  # introduce the randomness to against overfitting
                'colsample_bytree': 1,
                'gamma': 0.1,
              }

# all the regression will be saved in object dems, reffering to function regression_sf_sc
# and the regressor has been trained will be saved in the json file.
for dem,vx in zip(['dtm1','dtm10','cop30','fab'],[v_x_dtm1,v_x_dtm10,v_x_cop30,v_x_fab]): #
      dems.regression_sf_sc(dem=dem, 
                        v_x=vx,
                        # use regressor_name to save the regressor, use regressor to load the regressor
                        regressor_name=f'{dem}_abserror_250_10_qc_geosegment_feb.json',
                        weight=None)

### Step 2.1 Optional Evaluation of Bias correction

We have a tiny dataset from field trip for evaluation (only 65 points of snow depth), which is not included in the Zenodo dataset.

In [ ]:

from xsnow.goregression import evaluate_bias_correction

# (1) Print the quantile of the snow depth at 0.1, 0.5, 0.995. 
# To check the negative value (which is not true for 'snow depth').
print(dems.sc['sd_correct_dtm1'].quantile(q=[0.1,0.5,0.995]))
print(dems.sc['snowdepth_dtm1'].quantile(q=[0.1,0.5,0.995]))

# (2) a tiny ALS ground truth dataset
p, df_sub_ = dems.validation_lidar(dem='dtm1',corrected_sd='sd_correct_dtm1',dst_res=(10,10)) # Use ground truth in resolution of 10m 
evaluate_bias_correction(dems.sc,df_sub_)

In [ ]:
import xsnow.goplot
from xsnow.goplot import plot_all_hist_sf
import matplotlib.pyplot as plt

# (3) Plot the histogram for the snow free condition, at national wide.
plot_all_hist_sf(dems.sf,std=None,perc_t=None,window=(-10,10))
plt.savefig('dtm1_cop30_hist_snow_free_.jpg',dpi=600)

## The analysis on the bias correction by SHAP

Figures in supplementary materials are generated by the following code.

## Step 3 Downwscaling Model training

We want to know the snow depth distribution in the area of interest. But we do only have snow depth retrieved from the step 2 (stored in object 'dems.sc').

In [ ]:
# The area of interest / time of interest
df_nve_08 = pd.read_csv('df_nve_08.csv')
df_nve_08['month'] = 4

bottom = df_nve_08.N.min()
top = df_nve_08.N.max()
left = df_nve_08.E.min()
right = df_nve_08.E.max()

# The following qc is for filtering training dataset.
dems.sc_qc = {'dtm1':f'({bottom-100000} < N < {top+100000}) & ({left-50000} < E < {right+50000})& sd_correct_dtm1 >=0 & n_te_photons > 30 & h_te_uncertainty < 25 & {dems.era} < 12 & abs(df_dtm1_era5) < 12',
              'dtm10':f'({bottom-100000} < N < {top+100000}) & ({left-50000} < E < {right+50000})& sd_correct_dtm10 >=0 & n_te_photons > 30 & h_te_uncertainty < 25 & {dems.era} < 12 & abs(df_dtm10_era5) < 12',
              'cop30':f'({bottom-100000} < N < {top+100000}) & ({left-50000} < E < {right+50000})& sd_correct_cop30 >=0 & n_te_photons > 30 & h_te_uncertainty < 25 & {dems.era} < 12 & abs(df_cop30_era5) < 12',
              'fab':f'({bottom-100000} < N < {top+100000}) & ({left-50000} < E < {right+50000})& sd_correct_fab >=0 & n_te_photons > 30 & h_te_uncertainty < 25 & {dems.era} < 12 & abs(df_fab_era5) < 12'}

In [ ]:
# The combination of sc and sf. We combine it because we want the model are traning for all season dataset, not just for specific month. 
# Note that the sc and sf should have the same columns. So the following code is to make sure that the columns are the same.

# Iterate through each DEM
for dem_name in ['dtm1', 'dtm10', 'cop30', 'fab']:
    # Calculate the difference between the corrected snow depth and ERA5
    dems.sc[f'df_{dem_name}_era'] = dems.sc[f'sd_correct_{dem_name}'] - dems.sf[dems.era]

# combine with sf
# append new columns
dems.sf['snowdepth_cop30'] = dems.sf['snowdepth_fab'] = dems.sf['snowdepth_dtm1'] = dems.sf['snowdepth_dtm10'] = 0
dems.sc[['dh_after_dtm1','dh_after_dtm10','dh_after_fab','dh_after_cop30']] = dems.sc[['snowdepth_dtm1','snowdepth_dtm10','snowdepth_fab','snowdepth_cop30']]

# when df = sd_correct - sd_era. If you wanna correct sd_era, here shoule be the following
dems.sf['df_dtm1_era5'] = dems.sf['df_dtm10_era5'] = dems.sf['df_fab_era5'] = dems.sf['df_cop30_era5'] = 0 - dems.sf[dems.era]
dems.sf['sd_correct_dtm1'] = dems.sf['sd_correct_dtm10'] = dems.sf['sd_correct_cop30'] = dems.sf['sd_correct_fab'] = 0
dems.sf['subset_te_flag'] = 5
dems.sf['month'] = pd.DatetimeIndex(dems.sf['date_']).month
dems.sc['month'] = pd.DatetimeIndex(dems.sc['date_']).month

# Find shared columns
shared_columns = dems.sc.columns.intersection(dems.sf.columns)
# Combine and keep shared columns
combine_sc = pd.concat([dems.sc[shared_columns], dems.sf[shared_columns]], axis=0, ignore_index=True)
dems.sc = combine_sc

In [ ]:
# Parameters for the downscaling regression.
dems.params_2 = {
                'objective': 'reg:absoluteerror',
                'max_depth': 10,
                'n_estimators':350,
                'learning_rate': 0.1,
                'min_child_weight': 1,
                'subsample': 0.7,
                'colsample_bytree': 1,
                'gamma': 0.1,
                'verbose_eval':True,
            }
# Features for the downscaling regression.
v_x=['E','N','h_te_best_fit', 'slope', 'aspect', 'planc','profc',  # 'sde_se'
     'tpi','tpi_9','tpi_27','curvature','sde_era','wf_positive','wf_negative','month']


for dem in ['dtm1','dtm10','cop30','fab']:
     dems.regression_sc_sd_predict(dem=dem,
                                   raw_pts=df_nve_08,
                                   v_x=v_x,
                                   regressor_name=f'sd_{dem}_abserror_250_10_qc_nve_geosegment_abs_feb.json',
                                   regression='OTHERS',
                                   weight=None);

# Untill now, we finised the traning of the model. We can use it for production.

### 3.1 optional evaluation

In [ ]:
# Which snow depth from ERA has been donwscaled
print(dems.era)

# For 7127 points, we know the ground truth of snow depth from NVE's ALS. So that we can check the performance of the model.
from xsnow.goplot import plot_scater_nve
v_dtm1 =plot_scater_nve(df_nve_08.query('sd_nve_10 > 0 & sd_predict_dtm1 > -1'), 'sd_nve_10', 'sd_predict_dtm1', y_offset=1,title='10 m NVE')
v_dtm1


In [ ]:
# Also, we can check the performance of the model by using the histogram.
# Where we can see the distribution of the snow depth from NVE and the downscaling model.
# the downscaling model has much less extreame value

from xsnow.goregression import evaluate_downscaling
from xsnow.goplot import final_histogram
final_histogram(df_nve_08.query('sd_nve_10>0 & profc > -20')['sd_nve_10'],df_nve_08.query('sd_nve_10>0 & profc > -20')['sd_predict_dtm1']-1,dH_ref=df_nve_08.query('sd_nve_20>0')['sd_nve_20'],range=(0,8),window=(0,8),legend=['NVE 10','downscaling'],title='snow depth');
evaluate_downscaling(df_nve_08.query('sd_nve_10>0 & profc > -20'),cols=['sd_predict_dtm1'],offset=1);